In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.display import HTML

import pandas as pd
from decimal import Decimal
import datetime
from base64 import b64decode, b64encode
from django.db.models import Sum
from calendar import monthrange, month_name
import os
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.lib.pagesizes import letter, A4
# import temp_pdf_generate.py
from main import temp_pdf_generate

In [13]:
counter = 6
month = 12
year = 2020

counter_obj = Counter.objects.get(id=counter)
if counter != 24:
    both_counter_sale_obj = CounterEmployeeTraceMap.objects.filter(collection_date__month=month, collection_date__year=year, counter_id=counter)
    mor_counter_sale_obj = CounterEmployeeTraceMap.objects.filter(collection_date__month=month, collection_date__year=year, counter_id=counter, counteremployeetracesalegroupmap__icustomer_sale_group__session_id=1)
    eve_counter_sale_obj = CounterEmployeeTraceMap.objects.filter(collection_date__month=month, collection_date__year=year, counter_id=counter, counteremployeetracesalegroupmap__icustomer_sale_group__session_id=2)

    #employees 
    employee_list =list(both_counter_sale_obj.values_list('employee__user_profile__user__first_name', flat=True))
    employee_list = list(set(employee_list))

    #for sale data
    both_product_pkt_list = list(both_counter_sale_obj.values_list('collection_date', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))

    #mor
    mor_product_pkt_list = list(mor_counter_sale_obj.values_list('collection_date', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))
    mor_product_cost_list = list(mor_counter_sale_obj.values_list('collection_date', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__cost_for_month'))

    #eve
    eve_product_pkt_list = list(eve_counter_sale_obj.values_list('collection_date', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))
    eve_product_cost_list = list(eve_counter_sale_obj.values_list('collection_date', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__cost_for_month'))


else:
    both_counter_sale_obj = ICustomerSaleGroup.objects.filter(time_created__month=month, time_created__year=year, ordered_via_id__in=[1, 3])
    mor_counter_sale_obj = ICustomerSaleGroup.objects.filter(time_created__month=month, time_created__year=year, ordered_via_id__in=[1, 3], session_id=1)
    eve_counter_sale_obj = ICustomerSaleGroup.objects.filter(time_created__month=month, time_created__year=year, ordered_via_id__in=[1, 3], session_id=2)

    #employees
    employee_list = ['Online']

     #for sale data
    both_product_pkt_list = list(both_counter_sale_obj.values_list('time_created', 'icustomersale__product__short_name', 'icustomersale__count'))

    #mor
    mor_product_pkt_list = list(mor_counter_sale_obj.values_list('time_created', 'icustomersale__product__short_name', 'icustomersale__count'))
    mor_product_cost_list = list(mor_counter_sale_obj.values_list('time_created', 'icustomersale__count', 'icustomersale__cost_for_month'))

    #eve
    eve_product_pkt_list = list(eve_counter_sale_obj.values_list('time_created', 'icustomersale__product__short_name', 'icustomersale__count'))
    eve_product_cost_list = list(eve_counter_sale_obj.values_list('time_created', 'icustomersale__count', 'icustomersale__cost_for_month'))


#--------------------------for individual product total count moring + eve----------------------#
if not both_counter_sale_obj.exists():
    date_list = list(both_counter_sale_obj.values_list('collection_date', flat=True))
    both_product_pkt_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500'])
    for i in date_list:
        both_product_pkt_df = both_product_pkt_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0}, ignore_index=True)
else:
    product_obj = Product.objects.all()
    both_product_pkt_column = ['date', 'product_name', 'count']
    both_product_pkt_df = pd.DataFrame(both_product_pkt_list, columns=both_product_pkt_column)

    if counter == 24:
        for i in list(both_product_pkt_df['date']):
            both_product_pkt_df.loc[both_product_pkt_df['date'] == i, ['date']] = str(i)[:10]

    both_product_pkt_df = both_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum}).reset_index()

    #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
    both_product_pkt_df = pd.pivot_table(both_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

    #convert_pivot_table_to_normal_df
    both_product_pkt_df.columns = both_product_pkt_df.columns.droplevel(0) #remove amount
    both_product_pkt_df.columns.name = None  #remove categories
    both_product_pkt_df = both_product_pkt_df.reset_index() #index to columns

    for i in list(both_product_pkt_df.columns)[1:]:
        both_product_pkt_df[i] = round((both_product_pkt_df[i] * float(product_obj.get(short_name=i).quantity)) / 1000, 3)


#----------------------for morning counter sale---------------#
if not mor_counter_sale_obj.exists():
    date_list = list(both_counter_sale_obj.values_list('collection_date', flat=True))
    mor_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
    for i in date_list:
        mor_counter_df = mor_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)
else:
    #morning_counter_sale_product_pkt_df 
    mor_product_pkt_column = ['date', 'product_name', 'count']
    mor_product_pkt_df = pd.DataFrame(mor_product_pkt_list, columns=mor_product_pkt_column)

    if counter == 24:
        for i in list(mor_product_pkt_df['date']):
            mor_product_pkt_df.loc[mor_product_pkt_df['date'] == i, ['date']] = str(i)[:10]

    mor_product_pkt_df = mor_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum}).reset_index()

    #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
    mor_product_pkt_df = pd.pivot_table(mor_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

    #convert_pivot_table_to_normal_df
    mor_product_pkt_df.columns = mor_product_pkt_df.columns.droplevel(0) #remove amount
    mor_product_pkt_df.columns.name = None  #remove categories
    mor_product_pkt_df = mor_product_pkt_df.reset_index() #index to columns
    #--

    #morning_counter_sale_product_cost_df
    mor_product_cost_column = ['date', 'count', 'cost']
    mor_product_cost_df = pd.DataFrame(mor_product_cost_list, columns=mor_product_cost_column)

    if counter == 24:
        for i in list(mor_product_cost_df['date']):
            mor_product_cost_df.loc[mor_product_cost_df['date'] == i, ['date']] = str(i)[:10]

    mor_product_cost_df = mor_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
    #--

    mor_counter_df = pd.merge(mor_product_pkt_df, mor_product_cost_df, left_on='date', right_on='date', how='left')

    mor_counter_df

#----------------------for evening counter sale---------------#
if not eve_counter_sale_obj.exists():
    date_list = list(both_counter_sale_obj.values_list('collection_date', flat=True))
    eve_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
    for i in date_list:
        eve_counter_df = eve_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)

else:
    #evening_counter_sale_product_pkt_df 
    eve_product_pkt_column = ['date', 'product_name', 'count']
    eve_product_pkt_df = pd.DataFrame(eve_product_pkt_list, columns=eve_product_pkt_column)

    if counter == 24:
        for i in list(eve_product_pkt_df['date']):
            eve_product_pkt_df.loc[eve_product_pkt_df['date'] == i, ['date']] = str(i)[:10]

    eve_product_pkt_df = eve_product_pkt_df.groupby(['date', 'product_name']).agg({'count': 'sum'}).reset_index()

    #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
    eve_product_pkt_df = pd.pivot_table(eve_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

    #convert_pivot_table_to_normal_df
    eve_product_pkt_df.columns = eve_product_pkt_df.columns.droplevel(0) #remove amount
    eve_product_pkt_df.columns.name = None  #remove categories
    eve_product_pkt_df = eve_product_pkt_df.reset_index() #index to columns
    #--

    #evening_counter_sale_product_cost_df
    eve_product_cost_column = ['date', 'count', 'cost']
    eve_product_cost_df = pd.DataFrame(eve_product_cost_list, columns=eve_product_cost_column)

    if counter == 24:
        for i in list(eve_product_cost_df['date']):
            eve_product_cost_df.loc[eve_product_cost_df['date'] == i, ['date']] = str(i)[:10]

    eve_product_cost_df = eve_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
    #--

    eve_counter_df = pd.merge(eve_product_pkt_df, eve_product_cost_df, left_on='date', right_on='date', how='left')
    eve_counter_df = eve_counter_df.reindex(columns=list(mor_counter_df.columns)).fillna(0)

final_df = pd.merge(mor_counter_df, eve_counter_df, left_on='date', right_on='date', how='left').fillna(0)

final_df = pd.merge(final_df, both_product_pkt_df, left_on='date', right_on='date', how='left').fillna(0)

head_dict = {}
for i in list(final_df.columns):
    if i[-1] == 'x':
        head_dict[i] = i[:-1]+'mor'
    elif i[-1] == 'y':
        head_dict[i] = i[:-1]+'eve'
    elif i != 'date' and  i[-1] != 'x' and i[-1] != 'y':
        head_dict[i] = i+'_total'

final_df = final_df.rename(columns=head_dict)
final_df = final_df.append(final_df.sum(), ignore_index=True)
final_df.iloc[-1, final_df.columns.get_loc('date')] = 'Total'
final_df.index += 1

# # #-------------------------------------for_pdf----------------------------------------------------------------# 

# file_name = f"month_wise_counter_sale_for_{counter_obj.name} - ({month_name[month] +'-'+ str(year)})" + '.pdf'
# file_path = os.path.join('static/media/', file_name)
# mycanvas = canvas.Canvas(file_path, pagesize=(15 * inch, 12 * inch))

# # # ________Head_lines________#

# mycanvas.setFont('Helvetica', 15)
# mycanvas.drawCentredString(520, 840,
#                            'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
# mycanvas.setFont('Helvetica', 14)

# mycanvas.drawCentredString(520, 825, str(f"Monthly Conter Wise Report for - {month_name[month] +'-'+ str(year)}"))
# mycanvas.setFont('Helvetica-Bold', 10)
# mycanvas.setLineWidth(0)

# x_axis = 30
# y_axis = 765

# mycanvas.drawString(x_axis, y_axis+30 , 'SALES MAN NAME : '+ str(*employee_list))
# mycanvas.drawRightString(x_axis+1015, y_axis+30 , 'COUNTER : ' + f'{str(counter_obj.name)}')

# mycanvas.drawString(x_axis, y_axis, 'Sl No')
# mycanvas.line(x_axis-5, y_axis+20, x_axis-5, y_axis-30)
# mycanvas.drawString(x_axis+50, y_axis, 'Date')
# mycanvas.line(x_axis+30, y_axis+20, x_axis+30, y_axis-30)

# mycanvas.drawString(x_axis+160, y_axis, 'TM Packets')
# mycanvas.drawString(x_axis+120, y_axis-25, 'AM')
# mycanvas.drawString(x_axis+170, y_axis-25, 'PM')
# mycanvas.drawString(x_axis+215, y_axis-25, 'TOTAL Ltr')
# mycanvas.line(x_axis+100, y_axis+20, x_axis+100, y_axis-30)
# mycanvas.line(x_axis+270, y_axis+20, x_axis+270, y_axis-30)

# mycanvas.drawString(x_axis+320, y_axis, 'SM500 Packets')
# mycanvas.drawString(x_axis+290, y_axis-25, 'AM')
# mycanvas.drawString(x_axis+340, y_axis-25, 'PM')
# mycanvas.drawString(x_axis+385, y_axis-25, 'TOTAL Ltr')
# mycanvas.line(x_axis+440, y_axis+20, x_axis+440, y_axis-30)

# mycanvas.drawString(x_axis+490, y_axis, 'SM250 Packets')
# mycanvas.drawString(x_axis+460, y_axis-25, 'AM')
# mycanvas.drawString(x_axis+510, y_axis-25, 'PM')
# mycanvas.drawString(x_axis+555, y_axis-25, 'TOTAL Ltr')
# mycanvas.line(x_axis+610, y_axis+20, x_axis+610, y_axis-30)

# mycanvas.drawString(x_axis+660, y_axis, 'FCM500 Packets')
# mycanvas.drawString(x_axis+630, y_axis-25, 'AM')
# mycanvas.drawString(x_axis+680, y_axis-25, 'PM')
# mycanvas.drawString(x_axis+725, y_axis-25, 'TOTAL Ltr')
# mycanvas.line(x_axis+780, y_axis+20, x_axis+780, y_axis-30)

# mycanvas.drawString(x_axis+805, y_axis, 'Total Litre')
# # mycanvas.drawString(x_axis+800, y_axis-25, 'AM')
# # mycanvas.drawString(x_axis+850, y_axis-25, 'PM')
# mycanvas.line(x_axis+890, y_axis+20, x_axis+890, y_axis-30)

# mycanvas.drawString(x_axis+920, y_axis, 'Total Value')
# # mycanvas.drawString(x_axis+915, y_axis-25, 'AM')
# # mycanvas.drawString(x_axis+975, y_axis-25, 'PM')
# mycanvas.line(x_axis+1020, y_axis+20, x_axis+1020, y_axis-30)

# mycanvas.line(x_axis-5, y_axis+20, x_axis+1020, y_axis+20)
# mycanvas.line(x_axis-5, y_axis-10, x_axis+1020, y_axis-10)

# mycanvas.line(x_axis-5, y_axis-30, x_axis+1020, y_axis-30)

# for index,i in final_df.iterrows():
#     if i.date == 'Total':
#         date = "Total"
#     else:
#         date = i.date
#         if counter == 24:
#             date = datetime.datetime.strptime(date, '%Y-%m-%d')
#         date = datetime.datetime.strftime(date, '%d-%m-%Y')
#         mycanvas.drawRightString(x_axis+20,y_axis-50,str(index))
#     mycanvas.drawRightString(x_axis+95,y_axis-50,str(date))

#     mycanvas.drawRightString(x_axis+150,y_axis-50,str(round(i['TM 500_mor'])))
#     mycanvas.drawRightString(x_axis+200,y_axis-50,str(round(i['TM 500_eve'])))
#     mycanvas.drawRightString(x_axis+265,y_axis-50,str(round(Decimal(i['TM 500_total']), 3)))

#     mycanvas.drawRightString(x_axis+315,y_axis-50,str(round(i['STD 500_mor'])))
#     mycanvas.drawRightString(x_axis+370,y_axis-50,str(round(i['STD 500_eve'])))
#     mycanvas.drawRightString(x_axis+435,y_axis-50,str(round(Decimal(i['STD 500_total']), 3)))

#     mycanvas.drawRightString(x_axis+485,y_axis-50,str(round(i['STD 250_mor'])))
#     mycanvas.drawRightString(x_axis+535,y_axis-50,str(round(i['STD 250_eve'])))
#     mycanvas.drawRightString(x_axis+605,y_axis-50,str(round(Decimal(i['STD 250_total']), 3)))

#     mycanvas.drawRightString(x_axis+655,y_axis-50,str(round(i['FCM 500_mor'])))
#     mycanvas.drawRightString(x_axis+705,y_axis-50,str(round(i['FCM 500_eve'])))
#     mycanvas.drawRightString(x_axis+775,y_axis-50,str(round(Decimal(i['FCM 500_total']), 3)))

# #     mycanvas.drawRightString(x_axis+825,y_axis-50,str(round(i['count_mor'])))
#     mycanvas.drawRightString(x_axis+875,y_axis-50,str(round(Decimal(i['TM 500_total'] + i['STD 500_total'] + i['STD 250_total'] + i['FCM 500_total']), 3)))

# #     mycanvas.drawRightString(x_axis+945,y_axis-50,str(round(i['cost_mor'])))
#     mycanvas.drawRightString(x_axis+1005,y_axis-50,str(round(Decimal(i['cost_eve'] + i['cost_mor']), 2)))

#     #lines
#     mycanvas.line(x_axis-5, y_axis+20, x_axis-5, y_axis-60)
#     mycanvas.line(x_axis+30, y_axis+20, x_axis+30, y_axis-60)

#     mycanvas.line(x_axis+100, y_axis+20, x_axis+100, y_axis-60)
#     mycanvas.line(x_axis+160, y_axis-10, x_axis+160, y_axis-60)
#     mycanvas.line(x_axis+210, y_axis-10, x_axis+210, y_axis-60)

#     mycanvas.line(x_axis+270, y_axis+20, x_axis+270, y_axis-60)
#     mycanvas.line(x_axis+325, y_axis-10, x_axis+325, y_axis-60)
#     mycanvas.line(x_axis+380, y_axis-10, x_axis+380, y_axis-60)

#     mycanvas.line(x_axis+440, y_axis+20, x_axis+440, y_axis-60)
#     mycanvas.line(x_axis+495, y_axis-10, x_axis+495, y_axis-60)
#     mycanvas.line(x_axis+550, y_axis-10, x_axis+550, y_axis-60)

#     mycanvas.line(x_axis+610, y_axis+20, x_axis+610, y_axis-60)
#     mycanvas.line(x_axis+670, y_axis-10, x_axis+670, y_axis-60)
#     mycanvas.line(x_axis+720, y_axis-10, x_axis+720, y_axis-60)

#     mycanvas.line(x_axis+780, y_axis+20, x_axis+780, y_axis-60)
# #     mycanvas.line(x_axis+835, y_axis-10, x_axis+835, y_axis-60)

#     mycanvas.line(x_axis+890, y_axis+20, x_axis+890, y_axis-60)
# #     mycanvas.line(x_axis+955, y_axis-10, x_axis+955, y_axis-60)

#     mycanvas.line(x_axis+1020, y_axis+20, x_axis+1020, y_axis-60)

#     y_axis -= 22

# mycanvas.line(x_axis-5, y_axis-15, x_axis+1020, y_axis-15)
# mycanvas.line(x_axis-5, y_axis-37, x_axis+1020, y_axis-37)

# mycanvas.save()

In [14]:
final_df

,date,FCM 500_mor,STD 250_mor,STD 500_mor,TM 500_mor,count_mor,cost_mor,FCM 500_eve,STD 250_eve,STD 500_eve,TM 500_eve,count_eve,cost_eve,FCM 500_total,STD 250_total,STD 500_total,TM 500_total
1,2020-12-02,0,20,67,35,122.0,75562.50,0.0,0.0,0.0,0.0,0.0,0,0.0,5.00,33.5,17.5
2,2020-12-03,20,12,54,11,97.0,63953.00,0.0,0.0,0.0,0.0,0.0,0,10.0,3.00,27.0,5.5
3,2020-12-04,5,6,14,25,50.0,31201.50,0.0,0.0,1.0,0.0,1.0,697.50,2.5,1.50,7.5,12.5
4,2020-12-05,31,11,126,18,186.0,126511.00,0.0,1.0,1.0,0.0,2.0,1054.00,15.5,3.00,63.5,9.0
5,2020-12-07,0,4,44,4,52.0,34596.00,0.0,0.0,0.0,0.0,0.0,0,0.0,1.00,22.0,2.0
6,2020-12-08,0,19,98,26,143.0,91121.00,0.0,0.0,0.0,0.0,0.0,0,0.0,4.75,49.0,13.0
7,Total,56,72,403,119,650.0,422945.00,0.0,1.0,2.0,0.0,3.0,1751.50,28.0,18.25,202.5,59.5


In [132]:
counter_id_list = [4, 6, 7, 21, 25]
counter_list = list(Counter.objects.filter(id__in=counter_id_list).values('name', 'id'))
counter_list

[{'name': 'Gandhipuram - Card Sales', 'id': 6},
 {'name': 'Ramanathapuram Card Sales', 'id': 7},
 {'name': 'R.S puram - Card Sales 02', 'id': 21},
 {'name': 'R.S puram - Card Sales 01', 'id': 4},
 {'name': 'Sundarapuram Card Sales', 'id': 25}]

### Card salse report for customers

In [6]:
counter_id_list = [4, 6, 7, 21, 25]
date = '2020-12-02'

counter_employee_list = list(CounterEmployeeTraceMap.objects.filter(collection_date=date, counter_id__in=counter_id_list).values_list('counter__name', 'employee__user_profile__user__first_name'))
counter_employee_column = ['counter', 'employee']
counter_employee_df = pd.DataFrame(counter_employee_list, columns=counter_employee_column)
counter_employee_df = counter_employee_df.drop_duplicates()

counter_employee_df = counter_employee_df.groupby('employee').agg({'counter': list}).reset_index()
counter_employee_dict = counter_employee_df.groupby('employee').apply(lambda x:x.to_dict('r')[0]).to_dict()
counter_employee_dict

{'Anandhakumar': {'employee': 'Anandhakumar',
  'counter': ['Gandhipuram - Card Sales']},
 'Anthonyswamy': {'employee': 'Anthonyswamy',
  'counter': ['Ramanathapuram Card Sales']},
 'N.Sivakumar': {'employee': 'N.Sivakumar',
  'counter': ['Sundarapuram Card Sales']},
 'Vatharnisha': {'employee': 'Vatharnisha',
  'counter': ['Sundarapuram Card Sales']},
 'chinthamani': {'employee': 'chinthamani',
  'counter': ['R.S puram - Card Sales 02', 'R.S puram - Card Sales 01']}}

In [22]:
    counter_id_list = [4, 6, 7, 21, 25]
    date = '2020-12-02'

    counter_employee_list = list(CounterEmployeeTraceMap.objects.filter(collection_date=date, counter_id__in=counter_id_list).values_list('counter__name', 'employee__user_profile__user__first_name'))
    counter_employee_column = ['counter', 'employee']
    counter_employee_df = pd.DataFrame(counter_employee_list, columns=counter_employee_column)
    counter_employee_df = counter_employee_df.drop_duplicates()

    counter_employee_df = counter_employee_df.groupby('employee').agg({'counter': list}).reset_index()
    counter_employee_dict = counter_employee_df.groupby('employee').apply(lambda x:x.to_dict('r')[0]).to_dict()
    counter_employee_dict

    #---------------------------------------------------

    both_counter_sale_obj = CounterEmployeeTraceMap.objects.filter(collection_date=date, counter_id__in=counter_id_list)
    mor_counter_sale_obj = both_counter_sale_obj.filter(counteremployeetracesalegroupmap__icustomer_sale_group__session_id=1)
    eve_counter_sale_obj = both_counter_sale_obj.filter(counteremployeetracesalegroupmap__icustomer_sale_group__session_id=2)

    #------------morning_evening-----#
    if not both_counter_sale_obj.exists():
        counter_list = list(both_counter_sale_obj.values_list('counter__name', flat=True))
        both_product_pkt_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500'])
        for i in counter_list:
            both_product_pkt_df = both_product_pkt_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0}, ignore_index=True)
    else:
        product_obj = Product.objects.all()
        both_product_pkt_list = list(both_counter_sale_obj.values_list('employee__user_profile__user__first_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))
        both_product_pkt_column = ['date', 'product_name', 'count']
        both_product_pkt_df = pd.DataFrame(both_product_pkt_list, columns=both_product_pkt_column)
        both_product_pkt_df = both_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        both_product_pkt_df = pd.pivot_table(both_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        #convert_pivot_table_to_normal_df
        both_product_pkt_df.columns = both_product_pkt_df.columns.droplevel(0) #remove amount
        both_product_pkt_df.columns.name = None  #remove categories
        both_product_pkt_df = both_product_pkt_df.reset_index() #index to columns

        for i in list(both_product_pkt_df.columns)[1:]:
            both_product_pkt_df[i] = round((both_product_pkt_df[i] * float(product_obj.get(short_name=i).quantity)) / 1000, 3)

    #----------------------for morning counter sale---------------#

    if not mor_counter_sale_obj.exists():
        counter_list = list(both_counter_sale_obj.values_list('counter__name', flat=True))
        mor_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
        for i in counter_list:
            mor_counter_df = mor_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)
    else:
        #morning_counter_sale_product_pkt_df 
        mor_product_pkt_list = list(mor_counter_sale_obj.values_list('employee__user_profile__user__first_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))
        mor_product_pkt_column = ['date', 'product_name', 'count']
        mor_product_pkt_df = pd.DataFrame(mor_product_pkt_list, columns=mor_product_pkt_column)
        mor_product_pkt_df = mor_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        mor_product_pkt_df = pd.pivot_table(mor_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        #convert_pivot_table_to_normal_df
        mor_product_pkt_df.columns = mor_product_pkt_df.columns.droplevel(0) #remove amount
        mor_product_pkt_df.columns.name = None  #remove categories
        mor_product_pkt_df = mor_product_pkt_df.reset_index() #index to columns
        #--

        #morning_counter_sale_product_cost_df
        mor_product_cost_list = list(mor_counter_sale_obj.values_list('employee__user_profile__user__first_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__cost_for_month'))
        mor_product_cost_column = ['date', 'count', 'cost']
        mor_product_cost_df = pd.DataFrame(mor_product_cost_list, columns=mor_product_cost_column)
        mor_product_cost_df = mor_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
        #--

        mor_counter_df = pd.merge(mor_product_pkt_df, mor_product_cost_df, left_on='date', right_on='date', how='left')


    #----------------------for evening counter sale---------------#

    if not eve_counter_sale_obj.exists():
        counter_list = list(both_counter_sale_obj.values_list('counter__name', flat=True))
        eve_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
        for i in counter_list:
            eve_counter_df = eve_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)

    else:
        #evening_counter_sale_product_pkt_df 
        eve_product_pkt_list = list(eve_counter_sale_obj.values_list('employee__user_profile__user__first_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__product__short_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count'))
        eve_product_pkt_column = ['date', 'product_name', 'count']
        eve_product_pkt_df = pd.DataFrame(eve_product_pkt_list, columns=eve_product_pkt_column)
        eve_product_pkt_df = eve_product_pkt_df.groupby(['date', 'product_name']).agg({'count': 'sum'})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        eve_product_pkt_df = pd.pivot_table(eve_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        convert_pivot_table_to_normal_df
        eve_product_pkt_df.columns = eve_product_pkt_df.columns.droplevel(0) #remove amount
        eve_product_pkt_df.columns.name = None  #remove categories
        eve_product_pkt_df = eve_product_pkt_df.reset_index() #index to columns
        #--

        #evening_counter_sale_product_cost_df
        eve_product_cost_list = list(eve_counter_sale_obj.values_list('employee__user_profile__user__first_name', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__count', 'counteremployeetracesalegroupmap__icustomer_sale_group__icustomersale__cost_for_month'))
        eve_product_cost_column = ['date', 'count', 'cost']
        eve_product_cost_df = pd.DataFrame(eve_product_cost_list, columns=eve_product_cost_column)
        eve_product_cost_df = eve_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
        #--

        eve_counter_df = pd.merge(eve_product_pkt_df, eve_product_cost_df, left_on='date', right_on='date', how='left')
        eve_counter_df = eve_counter_df.reindex(columns=list(mor_counter_df.columns)).fillna(0)

    final_df = pd.merge(mor_counter_df, eve_counter_df, left_on='date', right_on='date', how='left').fillna(0)

    final_df = pd.merge(final_df, both_product_pkt_df, left_on='date', right_on='date', how='left').fillna(0)


    #---------------------------------------------------for online --------------------------------------#

    both_counter_sale_obj = ICustomerSaleGroup.objects.filter(time_created__date=date, ordered_via_id__in=[1, 3])
    mor_counter_sale_obj = both_counter_sale_obj.filter(session_id=1)
    eve_counter_sale_obj = both_counter_sale_obj.filter(session_id=2)

    counter_list = ['Online']

    #--------------------------for individual product total count moring + eve----------------------#
    if not both_counter_sale_obj.exists():
        both_product_pkt_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500'])
        for i in counter_list:
            both_product_pkt_df = both_product_pkt_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0}, ignore_index=True)
    else:
        product_obj = Product.objects.all()
        both_product_pkt_list = list(both_counter_sale_obj.values_list('session_id', 'icustomersale__product__short_name', 'icustomersale__count'))
        both_product_pkt_column = ['date', 'product_name', 'count']
        both_product_pkt_df = pd.DataFrame(both_product_pkt_list, columns=both_product_pkt_column)
        both_product_pkt_df['date'] = 'Online'

        both_product_pkt_df = both_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        both_product_pkt_df = pd.pivot_table(both_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        #convert_pivot_table_to_normal_df
        both_product_pkt_df.columns = both_product_pkt_df.columns.droplevel(0) #remove amount
        both_product_pkt_df.columns.name = None  #remove categories
        both_product_pkt_df = both_product_pkt_df.reset_index() #index to columns

        for i in list(both_product_pkt_df.columns)[1:]:
            both_product_pkt_df[i] = round((both_product_pkt_df[i] * float(product_obj.get(short_name=i).quantity)) / 1000, 3)


    #----------------------for morning counter sale---------------#
    #morning_counter_sale_product_pkt_df 
    if not mor_counter_sale_obj.exists():
        mor_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
        for i in counter_list:
            mor_counter_df = mor_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)
    else:
        mor_product_pkt_list = list(mor_counter_sale_obj.values_list('session_id', 'icustomersale__product__short_name', 'icustomersale__count'))
        mor_product_pkt_column = ['date', 'product_name', 'count']
        mor_product_pkt_df = pd.DataFrame(mor_product_pkt_list, columns=mor_product_pkt_column)
        mor_product_pkt_df['date'] = 'Online'
        mor_product_pkt_df = mor_product_pkt_df.groupby(['date', 'product_name']).agg({'count': sum})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        mor_product_pkt_df = pd.pivot_table(mor_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        #convert_pivot_table_to_normal_df
        mor_product_pkt_df.columns = mor_product_pkt_df.columns.droplevel(0) #remove amount
        mor_product_pkt_df.columns.name = None  #remove categories
        mor_product_pkt_df = mor_product_pkt_df.reset_index() #index to columns
        #--

        #morning_counter_sale_product_cost_df
        mor_product_cost_list = list(mor_counter_sale_obj.values_list('session_id', 'icustomersale__count', 'icustomersale__cost_for_month'))
        mor_product_cost_column = ['date', 'count', 'cost']
        mor_product_cost_df = pd.DataFrame(mor_product_cost_list, columns=mor_product_cost_column)
        mor_product_cost_df['date'] = 'Online'
        mor_product_cost_df = mor_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
        #--

        mor_counter_df = pd.merge(mor_product_pkt_df, mor_product_cost_df, left_on='date', right_on='date', how='left')

    #----------------------for evening counter sale---------------#
    #evening_counter_sale_product_pkt_df 
    if not eve_counter_sale_obj.exists():
        eve_counter_df = pd.DataFrame(columns=['date', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'count', 'cost'])
        for i in counter_list:
            eve_counter_df = eve_counter_df.append({'date': i, 'FCM 500':0, 'STD 250':0, 'STD 500':0, 'TM 500':0, 'count': 0, 'cost': 0}, ignore_index=True)
    else:
        eve_product_pkt_list = list(eve_counter_sale_obj.values_list('session_id', 'icustomersale__product__short_name', 'icustomersale__count'))
        eve_product_pkt_column = ['date', 'product_name', 'count']
        eve_product_pkt_df = pd.DataFrame(eve_product_pkt_list, columns=eve_product_pkt_column)
        eve_product_pkt_df['date'] = 'Online'
        eve_product_pkt_df = eve_product_pkt_df.groupby(['date', 'product_name']).agg({'count': 'sum'})

        #convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
        eve_product_pkt_df = pd.pivot_table(eve_product_pkt_df, index='date', columns='product_name', aggfunc=min, fill_value=0)

        #convert_pivot_table_to_normal_df
        eve_product_pkt_df.columns = eve_product_pkt_df.columns.droplevel(0) #remove amount
        eve_product_pkt_df.columns.name = None  #remove categories
        eve_product_pkt_df = eve_product_pkt_df.reset_index() #index to columns
        #--

        #evening_counter_sale_product_cost_df
        eve_product_cost_list = list(eve_counter_sale_obj.values_list('session_id', 'icustomersale__count', 'icustomersale__cost_for_month'))
        eve_product_cost_column = ['date', 'count', 'cost']
        eve_product_cost_df = pd.DataFrame(eve_product_cost_list, columns=eve_product_cost_column)
        eve_product_cost_df['date'] = 'Online'
        eve_product_cost_df = eve_product_cost_df.groupby('date').agg({'count': 'sum', 'cost': 'sum'}).reset_index()
        #--

        eve_counter_df = pd.merge(eve_product_pkt_df, eve_product_cost_df, left_on='date', right_on='date', how='left')
        eve_counter_df = eve_counter_df.reindex(columns=list(mor_counter_df.columns)).fillna(0)

    final_df2 = pd.merge(mor_counter_df, eve_counter_df, left_on='date', right_on='date', how='left').fillna(0)
    final_df2 = pd.merge(final_df2, both_product_pkt_df, left_on='date', right_on='date', how='left').fillna(0)

    #final_df
    final_df = pd.concat([final_df, final_df2], ignore_index=True)
    head_dict = {}
    for i in list(final_df.columns):
        if i[-1] == 'x':
            head_dict[i] = i[:-1]+'mor'
        elif i[-1] == 'y':
            head_dict[i] = i[:-1]+'eve'
        elif i != 'date' and  i[-1] != 'x' and i[-1] != 'y':
            head_dict[i] = i+'_total'

    final_df = final_df.rename(columns=head_dict)
    final_df = final_df.append(final_df.sum(), ignore_index=True)
    final_df.iloc[-1, final_df.columns.get_loc('date')] = 'Total'
    final_df.index += 1


    # # #-------------------------------------for_pdf----------------------------------------------------------------# 

    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    date = datetime.datetime.strftime(date, '%d-%m-%Y')

    file_name = f"date_wise_counter_sale_for({date})" + '.pdf'
    file_path = os.path.join('static/media/', file_name)
    mycanvas = canvas.Canvas(file_path, pagesize=(15 * inch, 12 * inch))

    # # ________Head_lines________#

    mycanvas.setFont('Helvetica', 15)
    mycanvas.drawCentredString(520, 840,
                               'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
    mycanvas.setFont('Helvetica', 14)

    mycanvas.drawCentredString(520, 825, str(f"Daily Conter Wise Report for - {date}"))
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.setLineWidth(0)

    x_axis = 30
    y_axis = 745

    # mycanvas.drawString(x_axis, y_axis+30 , 'SALES MAN NAME : '+ str(*employee_list))
    # mycanvas.drawRightString(x_axis+1015, y_axis+30 , 'COUNTER : ' + f'{str(counter_obj.name)}')

    mycanvas.drawString(x_axis, y_axis, 'Sl No')
    mycanvas.line(x_axis-5, y_axis+20, x_axis-5, y_axis-30)
    mycanvas.drawString(x_axis+50, y_axis, 'Date')
    mycanvas.line(x_axis+30, y_axis+20, x_axis+30, y_axis-30)

    mycanvas.drawString(x_axis+160, y_axis, 'TM Packets')
    mycanvas.drawString(x_axis+120, y_axis-25, 'AM')
    mycanvas.drawString(x_axis+170, y_axis-25, 'PM')
    mycanvas.drawString(x_axis+220, y_axis-25, 'TOTAL')
    mycanvas.line(x_axis+100, y_axis+20, x_axis+100, y_axis-30)
    mycanvas.line(x_axis+270, y_axis+20, x_axis+270, y_axis-30)

    mycanvas.drawString(x_axis+320, y_axis, 'SM500 Packets')
    mycanvas.drawString(x_axis+290, y_axis-25, 'AM')
    mycanvas.drawString(x_axis+340, y_axis-25, 'PM')
    mycanvas.drawString(x_axis+390, y_axis-25, 'TOTAL')
    mycanvas.line(x_axis+440, y_axis+20, x_axis+440, y_axis-30)

    mycanvas.drawString(x_axis+490, y_axis, 'SM250 Packets')
    mycanvas.drawString(x_axis+460, y_axis-25, 'AM')
    mycanvas.drawString(x_axis+510, y_axis-25, 'PM')
    mycanvas.drawString(x_axis+560, y_axis-25, 'TOTAL')
    mycanvas.line(x_axis+610, y_axis+20, x_axis+610, y_axis-30)

    mycanvas.drawString(x_axis+660, y_axis, 'FCM500 Packets')
    mycanvas.drawString(x_axis+630, y_axis-25, 'AM')
    mycanvas.drawString(x_axis+680, y_axis-25, 'PM')
    mycanvas.drawString(x_axis+730, y_axis-25, 'TOTAL')
    mycanvas.line(x_axis+780, y_axis+20, x_axis+780, y_axis-30)

    mycanvas.drawString(x_axis+805, y_axis, 'Total Litre')
    # mycanvas.drawString(x_axis+800, y_axis-25, 'AM')
    # mycanvas.drawString(x_axis+850, y_axis-25, 'PM')
    mycanvas.line(x_axis+890, y_axis+20, x_axis+890, y_axis-30)

    mycanvas.drawString(x_axis+920, y_axis, 'Total Value')
    # mycanvas.drawString(x_axis+915, y_axis-25, 'AM')
    # mycanvas.drawString(x_axis+975, y_axis-25, 'PM')
    mycanvas.line(x_axis+1020, y_axis+20, x_axis+1020, y_axis-30)

    mycanvas.line(x_axis-5, y_axis+20, x_axis+1020, y_axis+20)
    mycanvas.line(x_axis-5, y_axis-10, x_axis+1020, y_axis-10)

    mycanvas.line(x_axis-5, y_axis-30, x_axis+1020, y_axis-30)

    for index,i in final_df.iterrows():

        if i.date == 'Total':
            date = "Total"
        else:
            date = i.date
            mycanvas.drawRightString(x_axis+20,y_axis-50,str(index))

        mycanvas.setFont('Helvetica', 9)
        mycanvas.drawString(x_axis+32,y_axis-50,str(date)[:13])
        mycanvas.setFont('Helvetica', 10)
        if i.date != 'Online' and i.date != 'Total':
            for counter in counter_employee_dict[i.date]['counter']: 
                counter1 = counter[:12]
                counter2 = counter[13:26] + ','
                if counter == counter_employee_dict[i.date]['counter'][-1]:
                    counter2 = counter2[:-1] + ')'
                if counter == counter_employee_dict[i.date]['counter'][0]:
                    counter1 = '('+counter1
                mycanvas.drawString(x_axis+32,y_axis-70,str(counter1))
                y_axis -= 20
                mycanvas.drawString(x_axis+32,y_axis-70,str(counter2))
                y_axis -= 20
        mycanvas.drawRightString(x_axis+150,y_axis-50,str(round(i['TM 500_mor'])))
        mycanvas.drawRightString(x_axis+200,y_axis-50,str(round(i['TM 500_eve'])))
        mycanvas.drawRightString(x_axis+265,y_axis-50,str(round(Decimal(i['TM 500_total']), 3)))

        mycanvas.drawRightString(x_axis+315,y_axis-50,str(round(i['STD 500_mor'])))
        mycanvas.drawRightString(x_axis+370,y_axis-50,str(round(i['STD 500_eve'])))
        mycanvas.drawRightString(x_axis+435,y_axis-50,str(round(Decimal(i['STD 500_total']), 3)))

        mycanvas.drawRightString(x_axis+485,y_axis-50,str(round(i['STD 250_mor'])))
        mycanvas.drawRightString(x_axis+535,y_axis-50,str(round(i['STD 250_eve'])))
        mycanvas.drawRightString(x_axis+605,y_axis-50,str(round(Decimal(i['STD 250_total']), 3)))

        mycanvas.drawRightString(x_axis+655,y_axis-50,str(round(i['FCM 500_mor'])))
        mycanvas.drawRightString(x_axis+705,y_axis-50,str(round(i['FCM 500_eve'])))
        mycanvas.drawRightString(x_axis+775,y_axis-50,str(round(Decimal(i['FCM 500_total']), 3)))

    #     mycanvas.drawRightString(x_axis+825,y_axis-50,str(round(i['count_mor'])))
        mycanvas.drawRightString(x_axis+875,y_axis-50,str(round(Decimal(i['TM 500_total'] + i['STD 500_total'] + i['STD 250_total'] + i['FCM 500_total']), 3)))

    #     mycanvas.drawRightString(x_axis+945,y_axis-50,str(round(i['cost_mor'])))
        mycanvas.drawRightString(x_axis+1005,y_axis-50,str(round(Decimal(i['cost_eve'] + i['cost_mor']), 2)))

        #lines
        mycanvas.line(x_axis-5, y_axis+60, x_axis-5, y_axis-60)
        mycanvas.line(x_axis+30, y_axis+60, x_axis+30, y_axis-60)

        mycanvas.line(x_axis+100, y_axis+60, x_axis+100, y_axis-60)
        mycanvas.line(x_axis+160, y_axis+60, x_axis+160, y_axis-60)
        mycanvas.line(x_axis+210, y_axis+60, x_axis+210, y_axis-60)

        mycanvas.line(x_axis+270, y_axis+20, x_axis+270, y_axis-60)
        mycanvas.line(x_axis+325, y_axis-10, x_axis+325, y_axis-60)
        mycanvas.line(x_axis+380, y_axis-10, x_axis+380, y_axis-60)

        mycanvas.line(x_axis+440, y_axis+20, x_axis+440, y_axis-60)
        mycanvas.line(x_axis+495, y_axis-10, x_axis+495, y_axis-60)
        mycanvas.line(x_axis+550, y_axis-10, x_axis+550, y_axis-60)

        mycanvas.line(x_axis+610, y_axis+20, x_axis+610, y_axis-60)
        mycanvas.line(x_axis+670, y_axis-10, x_axis+670, y_axis-60)
        mycanvas.line(x_axis+720, y_axis-10, x_axis+720, y_axis-60)

        mycanvas.line(x_axis+780, y_axis+20, x_axis+780, y_axis-60)
    #     mycanvas.line(x_axis+835, y_axis-10, x_axis+835, y_axis-60)

        mycanvas.line(x_axis+890, y_axis+20, x_axis+890, y_axis-60)
    #     mycanvas.line(x_axis+955, y_axis-10, x_axis+955, y_axis-60)

        mycanvas.line(x_axis+1020, y_axis+20, x_axis+1020, y_axis-60)

        y_axis -= 42

    mycanvas.line(x_axis-5, y_axis-15, x_axis+1020, y_axis-15)
    mycanvas.line(x_axis-5, y_axis-37, x_axis+1020, y_axis-37)

    mycanvas.save()

/home/suneshrajan/Software/Virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [124]:
mor_product_cost_df

,date,count,cost
0,Online,5.0,3053.50


In [127]:
1. serve_monthly_counter_wise_report
2. serve_counter_employe_sale_for_date

NameError: name 'serve_counter_employe_sale_for_date' is not defined